In [1]:
import sys
sys.path.append('..')
import pyarrow.compute as pc
import pyarrow
import spines as ss
from pyarrow import Table as pt

ModuleNotFoundError: No module named 'pyarrow'

In [2]:
import numpy as np
import pandas as pd

test_fp = '/Users/guobingming/Downloads/ads_mdap_user_lsmd_gbm_model_train_sample_with_lag_df_new_0713.tsv'

In [3]:
df = ss.dataloader(test_fp, sep='\t', transform2low_mem=False) 

In [79]:
df = pd.concat([df for i in range(100)], axis=0, ignore_index=True)

In [4]:
from spines import *


def df_preview(dataset, indicators=None):
    """
    For data previews, to check various data properties of the dataset.
    returns:
    total: number of elements
    na: null values
    naPercent: null value accounts for this ratio
    nunique: unique values number
    dtype: datatype
    75%: 75% quantile
    25%: 25% quantile
    variation: variation ratio
    std: standard deviation
    skew: Skewness
    kurt: Kurtosis
    samples: Random returns two values
    """
    assert (isinstance(indicators, (list, tuple)) and len(indicators) > 0) or indicators is None

    available_indicators = (
        'total', 'na', 'naPercent', 'nunique', 'dtype', 'max', '75%', 'median',
        '25%', 'min', 'mean', 'mode', 'variation', 'std', 'skew', 'kurt', 'samples'
    )

    if indicators is not None:
        unavailable_indicators = []
        for i in indicators:
            if i not in available_indicators:
                unavailable_indicators.append(i)

        if len(unavailable_indicators) > 0:
            raise ValueError(f"{','.join(unavailable_indicators)} is unavailable indicators")

    col = dataset.columns
    num_cols = select_numeric_cols(dataset)

    ind_len = dataset.shape[0]

    dtypes = dataset.dtypes.to_dict()

    if indicators is None:
        indicators = available_indicators

    df = pd.DataFrame(columns=indicators)
    random_idx = np.random.randint(ind_len, size=2)

    global_params = {'total': ind_len}

    for i in col:
        i_data = dataset[i]

        if any([True if ind in indicators else False for ind in
                ['na', 'naPercent', 'nunique', 'max', '75%', 'median',
                 '25%', 'min', 'mean', 'mode', 'variation', 'std']
                ]):
            desc = i_data.describe()

            global_params['nunique'] = i_data.nunique()
            global_params['max'] = desc.get('max')
            global_params['75%'] = desc.get('75%')
            global_params['median'] = desc.get('50%')
            global_params['25%'] = desc.get('25%')
            global_params['min'] = desc.get('min')
            global_params['mean'] = desc.get('mean')
            global_params['std'] = desc.get('std')

            if 'na' in indicators or 'naPercent' in indicators:
                na_sum = ind_len - desc['count']

                global_params['na'] = na_sum
                global_params['naPercent'] = na_sum / ind_len

            if 'mode' in indicators or 'variation' in indicators:
                if not desc.get('top'):
                    _ = i_data.value_counts(ascending=False)
                    mode = _.index[0]
                    mode_count = _.iloc[0]
                else:
                    mode = desc['top']
                    mode_count = desc['freq']

                global_params['mode'] = mode
                global_params['mode_count'] = mode_count
                global_params['variation'] = 1 - mode_count / ind_len

        if 'dtype' in indicators:
            global_params['dtype'] = dtypes[i]

        if 'samples' in indicators:
            global_params['samples'] = tuple([i_data[idx] for idx in random_idx])

        if 'skew' in indicators:
            global_params['skew'] = i_data.skew() if i in num_cols else np.nan

        if 'kurt' in indicators:
            global_params['kurt'] = i_data.kurt() if i in num_cols else np.nan

        df.loc[i] = pd.Series({k: global_params[k] for k in indicators}, name=i)

    if df.shape[0] < df.shape[1]:
        return df.T
    else:
        return df

In [86]:
test = ss.timing_decorator(df_preview2)

In [87]:
test(df)

Function df_preview2 took 104.56514096260071 seconds to run.


,total,na,naPercent,nunique,dtype,max,75%,median,25%,min,mean,mode,variation,std,skew,kurt,samples
dt,10489500,0,0.0,105,object,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-19,0.990476,NaN,NaN,NaN,"(2023-02-19, 2023-01-03)"
userid,10489500,0,0.0,95386,object,NaN,NaN,NaN,NaN,NaN,NaN,c4dd38ce82524f1c9d5fedb4f57d75dd,0.999952,NaN,NaN,NaN,"(77c73a226f6345b2bde8622319d0e95f, f8b7f910433..."
last_category,10489500,0,0.0,99,object,NaN,NaN,NaN,NaN,NaN,NaN,drama_detail_page,0.447943,NaN,NaN,NaN,"(tvbsearchviewcontroller, drama_detail_page)"
vip_end_days,10489500,0,0.0,1083,int64,27652.0,81.000000,30.000000,10.000000,0.000000,78.391849,0,0.889899,169.459413,49.379494,6937.119681,"(9, 0)"
vip_effective_type,10489500,0,0.0,10,object,NaN,NaN,NaN,NaN,NaN,NaN,连续付费月卡,0.767034,NaN,NaN,NaN,"(端外付费, 其他)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lost_7_remain_days_td,10489500,0,0.0,310,int64,309.0,175.000000,97.000000,34.000000,0.000000,107.943696,6,0.982115,80.82563,0.359762,-1.041067,"(4, 196)"
lost_15_remain_days_td,10489500,0,0.0,302,int64,301.0,167.000000,89.000000,26.000000,0.000000,101.349216,14,0.984451,79.096753,0.412011,-1.030002,"(12, 188)"
next_cumulative_login_ds,10489500,0,0.0,316,int64,319.0,187.000000,111.000000,47.000000,4.000000,119.822136,5,0.990943,80.816423,0.309756,-1.043941,"(6, 206)"
next_login_7ds_freq,10489500,0,0.0,1803,float64,5.0,1.115385,1.044643,1.017544,1.006309,1.112636,2.0,0.988226,0.198254,4.607438,32.359641,"(1.5, 1.0098039215686274)"


In [82]:
test2 = ss.timing_decorator(ss.df_preview)
test2(df)

Function df_preview took 97.13236093521118 seconds to run.


,total,na,naPercent,nunique,dtype,max,75%,median,25%,min,mean,mode,variation,std,skew,kurt,samples
dt,10489500,0,0.0,105,object,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-19,0.990476,NaN,NaN,NaN,"(2023-04-10, 2023-01-27)"
userid,10489500,0,0.0,95386,object,NaN,NaN,NaN,NaN,NaN,NaN,c4dd38ce82524f1c9d5fedb4f57d75dd,0.999952,NaN,NaN,NaN,"(6ab7462f184342e5968c6ffe952c00c2, e0da13e663a..."
last_category,10489500,0,0.0,99,object,NaN,NaN,NaN,NaN,NaN,NaN,drama_detail_page,0.447943,NaN,NaN,NaN,"(drama_detail_page, home_page)"
vip_end_days,10489500,0.0,0.0,1083,int64,27652.0,81.0,30.0,10.0,0.0,78.391849,0,0.889899,169.459421,49.379494,6937.119681,"(268, 42)"
vip_effective_type,10489500,0,0.0,10,object,None,None,None,None,None,None,连续付费月卡,0.767034,None,NaN,NaN,"(端外付费, 连续付费季卡)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lost_7_remain_days_td,10489500,0.0,0.0,310,int64,309.0,175.0,97.0,34.0,0.0,107.943696,6,0.982115,80.825634,0.359762,-1.041067,"(41, 10)"
lost_15_remain_days_td,10489500,0.0,0.0,302,int64,301.0,167.0,89.0,26.0,0.0,101.349216,14,0.984451,79.096757,0.412011,-1.030002,"(33, 2)"
next_cumulative_login_ds,10489500,0.0,0.0,316,int64,319.0,187.0,111.0,47.0,4.0,119.822136,5,0.990943,80.816427,0.309756,-1.043941,"(57, 21)"
next_login_7ds_freq,10489500,0.0,0.0,1803,float64,5.0,1.115385,1.044643,1.017544,1.006309,1.112636,2.0,0.988226,0.198254,4.607438,32.359641,"(1.163265306122449, 1.1666666666666667)"


In [72]:
%prun df_preview2(df)

         1704119 function calls (1686080 primitive calls) in 1.779 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1435    0.358    0.000    0.358    0.000 {method 'call' of 'pyarrow._compute.Function' objects}
      217    0.239    0.001    0.243    0.001 algorithms.py:1010(value_counts_arraylike)
       93    0.095    0.001    0.095    0.001 {method 'unique' of 'pandas._libs.hashtable.Float64HashTable' objects}
     5868    0.060    0.000    0.060    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      865    0.042    0.000    0.042    0.000 {method 'argsort' of 'numpy.ndarray' objects}
      112    0.039    0.000    0.039    0.000 {method 'unique' of 'pandas._libs.hashtable.Int64HashTable' objects}
      205    0.039    0.000    0.081    0.000 nanops.py:1155(nanskew)
      205    0.038    0.000    0.080    0.000 nanops.py:1243(nankurt)
       11    0.035    0.003    0.035    0.003 {method 'unique' of 'pandas._libs

In [8]:
ss.df_preview(df)

,total,na,naPercent,nunique,dtype,max,75%,median,25%,min,mean,mode,variation,std,skew,kurt,samples
dt,104895,0,0.0,105,object,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-19,0.990476,NaN,NaN,NaN,"(2023-01-02, 2023-03-23)"
userid,104895,0,0.0,95386,object,NaN,NaN,NaN,NaN,NaN,NaN,c4dd38ce82524f1c9d5fedb4f57d75dd,0.999952,NaN,NaN,NaN,"(5192cbd2478f4df393b0aaa3e2d21371, 13e2a32e1d0..."
last_category,104895,0,0.0,99,object,NaN,NaN,NaN,NaN,NaN,NaN,drama_detail_page,0.447943,NaN,NaN,NaN,"(drama_detail_page, drama_detail_page)"
vip_end_days,104895,0.0,0.0,1083,int64,27652.0,81.0,30.0,10.0,0.0,78.391849,0,0.889899,169.460221,49.380193,6937.447112,"(162, 14)"
vip_effective_type,104895,0,0.0,10,object,None,None,None,None,None,None,连续付费月卡,0.767034,None,NaN,NaN,"(端外付费, 端外付费)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lost_7_remain_days_td,104895,0.0,0.0,310,int64,309.0,175.0,97.0,34.0,0.0,107.943696,6,0.982115,80.826015,0.359767,-1.041059,"(58, 25)"
lost_15_remain_days_td,104895,0.0,0.0,302,int64,301.0,167.0,89.0,26.0,0.0,101.349216,14,0.984451,79.09713,0.412017,-1.029994,"(50, 17)"
next_cumulative_login_ds,104895,0.0,0.0,316,int64,319.0,187.0,111.0,47.0,4.0,119.822136,5,0.990943,80.816808,0.309761,-1.043934,"(75, 36)"
next_login_7ds_freq,104895,0.0,0.0,1803,float64,5.0,1.115385,1.044643,1.017544,1.006309,1.112636,2.0,0.988226,0.198255,4.607503,32.361225,"(1.1363636363636365, 1.0909090909090908)"


In [73]:
%timeit df_preview2(df)

1.35 s ± 17.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit ss.df_preview(df)
%timeit df_preview(df)
%timeit df_preview2(df)

1.44 s ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.44 s ± 8.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.29 s ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [115]:
df.describe()

,vip_end_days,is_bought_today,unwatching_days,watch_last_nums,daydiff,before_last_date,vod_play_pv,vod_cnt,vod_vv,vod_over_pv,...,next_login_weekdiff,next_daydiff,next_login_vip_valid_month,lost_7_remain_ds,lost_15_remain_ds,lost_7_remain_days_td,lost_15_remain_days_td,next_cumulative_login_ds,next_login_7ds_freq,ranks
count,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,...,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.000000,104895.00000
mean,78.391849,0.018199,2.621193,0.126307,810.793946,3.775213,9.790962,1.750980,8.589380,1.324010,...,-0.202302,809.377530,2.588804,6.558091,13.369350,107.943696,101.349216,119.822136,1.112656,500.00000
std,169.460221,0.133672,11.135771,0.406953,598.864430,39.807242,11.058591,2.257497,10.213019,3.583156,...,0.151726,598.881969,5.645641,2.768173,5.162853,80.826015,79.097130,80.816808,0.198257,288.38769
min,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.005859,1.00000
25%,10.000000,0.000000,1.000000,0.000000,292.000000,1.000000,3.000000,1.000000,2.000000,0.000000,...,-0.142822,291.000000,0.333252,8.000000,16.000000,34.000000,26.000000,47.000000,1.017578,250.00000
50%,30.000000,0.000000,1.000000,0.000000,687.000000,1.000000,6.000000,1.000000,5.000000,0.000000,...,-0.142822,686.000000,1.000000,8.000000,16.000000,97.000000,89.000000,111.000000,1.044922,500.00000
75%,81.000000,0.000000,1.000000,0.000000,1351.000000,1.000000,13.000000,2.000000,11.000000,1.000000,...,-0.142822,1349.000000,2.666016,8.000000,16.000000,175.000000,167.000000,187.000000,1.115234,750.00000
max,27652.000000,1.000000,307.000000,16.000000,1950.000000,1848.000000,60.000000,60.000000,60.000000,60.000000,...,-0.142822,1949.000000,921.500000,8.000000,16.000000,309.000000,301.000000,319.000000,5.000000,999.00000
